In [59]:
# # dependency installation
# !pip install typing_extensions==4.6.1 pydantic pydantic-core cuml umap-learn pandas numpy matplotlib scikit-learn nltk langdetect bertopic[spacy] openai
# !pip install bertopic flair

In [127]:
# Load dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import download
import nltk
from nltk.corpus import stopwords
from wordcloud import WordCloud
import json
from nltk.stem import WordNetLemmatizer
import os

from bertopic.representation import KeyBERTInspired
from bertopic.representation import PartOfSpeech
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from bertopic import BERTopic
from bertopic.vectorizers import ClassTfidfTransformer
from bertopic.representation import MaximalMarginalRelevance
from bertopic.representation import OpenAI
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import DBSCAN
from sentence_transformers import SentenceTransformer
from wordcloud import WordCloud
import umap
import matplotlib.pyplot as plt
import openai
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel
from bertopic import BERTopic  
from sklearn.metrics import silhouette_score

#find stop words, along with recipe realted stopwords
stop = stopwords.words('english')

FinancialLabel = "Financial/Job Problem"
Legallabel = "Legal Problem"
SchoolLabel = "School or Academic Related Problem"
HealthLabel = "Lack of Access to Health/Mental Health Care"
FinancialPredicted = "Financial - Predicted"
LegalPredicted = "Legal - Predicted"
SchoolPredicted = "School - Predicted"
HealthPredicted = "Health - Predicted"


## Topic Modeling

In [128]:
# Pile all comments into single array

fix = HealthLabel
col = HealthPredicted

data = pd.read_csv('final working dataset with only suicide articles.csv')
data = data[data[fix] > 0]
comments_all = data['Only Reason']
print("before: ",len(comments_all))
comments_all = comments_all.dropna()

print("after: ",len(comments_all))

before:  3756
after:  3756


In [129]:
len(comments_all)
comments_all = comments_all.to_list()
len(comments_all)

3756

In [130]:
# seed topics
SEED_TOPICS_FLAG = False # seeds topics that model will attempt to converge to
# hdbscan params
MIN_CLUSTER_SIZE = 15

PRETRAINED_SENTENCE_MODEL = 'all-mpnet-base-v2'

In [131]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [132]:
sentence_model = SentenceTransformer(PRETRAINED_SENTENCE_MODEL)
embeddings = sentence_model.encode(comments_all, show_progress_bar=True)

Batches: 100%|██████████| 118/118 [02:01<00:00,  1.03s/it]


In [106]:
# init model
stopwords = stopwords.words('english')
umap_model = umap.UMAP(n_neighbors=10, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)
vectorizer_model = CountVectorizer(stop_words=stopwords, ngram_range=(1,2))
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)
representation_model = KeyBERTInspired()

In [100]:
#for siz in range(6, 7):

model = BERTopic(top_n_words = 10,
                    n_gram_range = (1,2),
                    language="english",
                    calculate_probabilities = True,
                    nr_topics=40,
                    embedding_model = PRETRAINED_SENTENCE_MODEL,
                    representation_model=representation_model,
                    vectorizer_model=vectorizer_model,
                    umap_model=umap_model)

topics, probs = model.fit_transform(comments_all, embeddings)

cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embedding1 = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embedding1, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)
print(" Number of topics: ",len(set(topics))-1)


# a1 = []
# a2 =[]
# a3 =[]
# a4 = []

# for a in range(len(topicsizes)):
#     for b in range(len(topic_num)):
#         coherencev, coherence, score, lentopics = func(topicsizes[a], topic_num[b])
#         a1.append(coherencev)
#         a2.append(coherence)
#         a3.append(score)
#         a4.append(lentopics)
tit = "All - " + fix
fig = model.visualize_barchart(top_n_topics=100, title = tit)
fig


KeyboardInterrupt: 

In [45]:
model.merge_topics(comments_all, [-1, 14])

In [ ]:
# Reduce the number of topics  
model.reduce_topics(comments_all, nr_topics="auto")
topics = model.topics_ 
model.visualize_barchart(top_n_topics=100, title = tit)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.46630358695983887, 0.48212456703186035, 0.49774977564811707,
                    0.5567287802696228, 0.5991394519805908],
              'xaxis': 'x',
              'y': [suicidal  , committed suicide  , suicide  , attempted suicide
                    , suicide article  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.49908876419067383, 0.5180374979972839, 0.526097297668457,
                    0.570652425289154, 0.7198272347450256],
              'xaxis': 'x2',
              'y': [suicides related  , among veterans  , veterans article  ,
                    suicide among  , veteran suicide  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.44919171929359436, 0.4587870240211487, 0.4623335301876068,
                    0.47081586718559265, 0.6091172695159912],
              'xaxis': 'x3',
              'y': [prison article  , suicide  , mentions inmates  , suicides  ,
                    solitary confinement  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.476432204246521, 0.5048592686653137, 0.5243644118309021,
                    0.5310384035110474, 0.5342926979064941],
              'xaxis': 'x4',
              'y': [suicides  , attempted suicide  , psychosis  , mental health  ,
                    postpartum psychosis  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5640214085578918, 0.5714514255523682, 0.5746006369590759,
                    0.575525164604187, 0.6558021306991577],
              'xaxis': 'x5',
              'y': [among transgender  , transgender people  , transgender
                    children  , transgender individuals  , transgender youth  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5218032002449036, 0.5530200004577637, 0.5582772493362427,
                    0.5643173456192017, 0.6162269115447998],
              'xaxis': 'x6',
              'y': [suicidal thoughts  , suicide rates  , suicide article  ,
                    discusses suicide  , suicide prevention  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5040199160575867, 0.5327291488647461, 0.5618453621864319,
                    0.5623766183853149, 0.5705956220626831],
              'xaxis': 'x7',
              'y': [brain damage  , encephalopathy cte  , concussions  , traumatic
                    encephalopathy  , encephalopathy  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.44640401005744934, 0.49289774894714355, 0.4998737573623657,
                    0.5135403275489807, 0.5295816659927368],
              'xaxis': 'x8',
              'y': [suicide  , killed surgeon  , killed patient  , suicide related
                    , suicide article  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.3896036744117737, 0.38993746042251587, 0.4115588068962097,
                    0.42027726769447327, 0.43756479024887085],
              'xaxis': 'x9',
              'y': [traumatic event  , suicide  , capitol attack  , capitol police
                    , took lives  ],
              'yaxis': 'y9'},
          

In [23]:
# Reduce the number of topics  
model.reduce_topics(comments_all, nr_topics=40)
topics = model.topics_ 
model.visualize_barchart(top_n_topics=100, title = tit)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.531216025352478, 0.5359881520271301, 0.5445011854171753,
                    0.591168999671936, 0.6756203174591064],
              'xaxis': 'x',
              'y': [mental health  , mental illness  , suicides  , suicide attempt
                    , suicide article  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5484499931335449, 0.5567085146903992, 0.558721661567688,
                    0.5652968287467957, 0.605559766292572],
              'xaxis': 'x2',
              'y': [attempted suicide  , suicides  , mental health  , suicide
                    rates  , discusses suicide  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4039679765701294, 0.4367721974849701, 0.4420381486415863,
                    0.4493408799171448, 0.5100884437561035],
              'xaxis': 'x3',
              'y': [mentions shooter  , purchase guns  , mass shooting  , suicide
                    , suicide article  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5289007425308228, 0.5573776364326477, 0.5654194355010986,
                    0.5929934978485107, 0.6152620315551758],
              'xaxis': 'x4',
              'y': [issue suicide  , discusses suicide  , among veterans  ,
                    suicide among  , suicides among  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4119448959827423, 0.4190005660057068, 0.42093464732170105,
                    0.43280357122421265, 0.4362477660179138],
              'xaxis': 'x5',
              'y': [cause suicide  , concussions  , encephalopathy  , mentions
                    adams  , phillip adams  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4589979648590088, 0.45979195833206177, 0.49515601992607117,
                    0.5077451467514038, 0.5087687969207764],
              'xaxis': 'x6',
              'y': [terminal cancer  , opioid addiction  , drug overdose  ,
                    overdoses  , terminal illness  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.3976864814758301, 0.39926570653915405, 0.41384997963905334,
                    0.4261173605918884, 0.4359435737133026],
              'xaxis': 'x7',
              'y': [suggests suicide  , mental health  , suicide  , traumatic
                    stress  , suicides  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4793442487716675, 0.4937523305416107, 0.5129443407058716,
                    0.5423488020896912, 0.5443166494369507],
              'xaxis': 'x8',
              'y': [shootings  , suicides  , gun violence  , mental health  , mass
                    shooting  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5640213489532471, 0.5714513659477234, 0.5746006965637207,
                    0.575525164604187, 0.6558021306991577],
              'xaxis': 'x9',
              'y': [among transgender  , transgender people  , transgender
                    children  , transgender individuals  , transgender youth  ],
              'yaxis': 'y9'},
             {'marker': {'color': '#CC79A7'},
      

In [139]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/safetensors/health6", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 2 - pytorch
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/pytorch/health6", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 3 - pickle
model.save("healthbertopic6", serialization="pickle")

2024-04-25 18:32:56,555 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [14]:
cleaned_docs = model._preprocess_text(comments_all)
vectorizer = model.vectorizer_model
analyzer = vectorizer.build_analyzer()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topicsForCS = model.get_topics()
topicsForCS.pop(-1, None)
topic_words = [[word for word, _ in model.get_topic(topic) if word != ''] for topic in topicsForCS]
#topic_words = [[words for words, _ in model.get_topic(topic)] for topic in range(len(set(topicsForCS))-1)]
final_topic_words = []
for i in topic_words:
    if len(i) > 0:
        final_topic_words.append(i)

# Evaluate
coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_v')
coherencev = coherence_model.get_coherence()

coherence_model = CoherenceModel(topics=final_topic_words, 
                    texts=tokens, 
                    corpus=corpus,
                    dictionary=dictionary, 
                    coherence='c_npmi')
coherence = coherence_model.get_coherence()
# Calculate embeddings for your documents  
embeddings = model._extract_embeddings(comments_all)  
# Calculate Silhouette score  
score = silhouette_score(embeddings, topics)  
print(" CV score: ",coherencev)
print(" C NPMI score: ",coherence)
print(" Silhouette score: ",score)

 CV score:  0.5496325968704261
 C NPMI score:  -0.17126519017716524
 Silhouette score:  0.021049919


In [74]:
# Reduce outliers
#new_topics = model.reduce_outliers(comments_all, topics, probabilities=probs, strategy="probabilities")
#model.update_topics(comments_all, topics=new_topics)

In [60]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/safetensors/health5", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 2 - pytorch
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model.save("/Users/minijain/Downloads/SocWebLab/Structural_Violence/pytorch/health5", serialization="pytorch", save_ctfidf=True, save_embedding_model=embedding_model)

# Method 3 - pickle
model.save("healthbertopic5", serialization="pickle")

2024-04-09 10:27:50,477 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [119]:
embedding_model = "sentence-transformers/" + PRETRAINED_SENTENCE_MODEL
model = BERTopic.load("healthbertopic5", embedding_model=embedding_model)

In [124]:
model.visualize_barchart(top_n_topics=100, title = tit)

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4155278205871582, 0.44413238763809204, 0.4442964196205139,
                    0.4534626305103302, 0.5090591907501221],
              'xaxis': 'x',
              'y': [suicides  , psychiatric  , committed suicide  , suicide  ,
                    suicide article  ],
              'yaxis': 'y'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.560097873210907, 0.5709260702133179, 0.5785031914710999,
                    0.5899217128753662, 0.642210841178894],
              'xaxis': 'x2',
              'y': [mental health  , mental illness  , suicide related  , suicide
                    attempt  , suicide article  ],
              'yaxis': 'y2'},
             {'marker': {'color': '#CC79A7'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5368539690971375, 0.5408509969711304, 0.5544157028198242,
                    0.6187630295753479, 0.6394949555397034],
              'xaxis': 'x3',
              'y': [suicidal thoughts  , attempted suicide  , suicide rates  ,
                    discusses suicide  , suicide prevention  ],
              'yaxis': 'y3'},
             {'marker': {'color': '#E69F00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.49908870458602905, 0.5180374979972839, 0.570652425289154,
                    0.6141881942749023, 0.7198272943496704],
              'xaxis': 'x4',
              'y': [suicides related  , among veterans  , suicide among  ,
                    suicides among  , veteran suicide  ],
              'yaxis': 'y4'},
             {'marker': {'color': '#56B4E9'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5039488077163696, 0.5053021907806396, 0.5230704545974731,
                    0.5429297685623169, 0.5552266240119934],
              'xaxis': 'x5',
              'y': [pandemic  , 19 pandemic  , suicides  , coronavirus pandemic  ,
                    covid 19  ],
              'yaxis': 'y5'},
             {'marker': {'color': '#009E73'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.4399995803833008, 0.4719892740249634, 0.47738116979599,
                    0.491260826587677, 0.49211785197257996],
              'xaxis': 'x6',
              'y': [substance abuse  , opioid addiction  , cause suicide  ,
                    overdoses  , drug overdose  ],
              'yaxis': 'y6'},
             {'marker': {'color': '#F0E442'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.5667771697044373, 0.573320746421814, 0.5775570273399353,
                    0.5776736736297607, 0.6588498950004578],
              'xaxis': 'x7',
              'y': [among transgender  , transgender people  , transgender
                    individuals  , transgender children  , transgender youth  ],
              'yaxis': 'y7'},
             {'marker': {'color': '#D55E00'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.6077508330345154, 0.644828736782074, 0.6834176182746887,
                    0.6901155710220337, 0.7071844339370728],
              'xaxis': 'x8',
              'y': [terminal illnesses  , euthanasia due  , euthanasia  , assisted
                    death  , assisted suicide  ],
              'yaxis': 'y8'},
             {'marker': {'color': '#0072B2'},
              'orientation': 'h',
              'type': 'bar',
              'x': [0.42093467712402344, 0.43280354142189026, 0.4362477660179138,
                    0.44091349840164185, 0.582173764705658],
              'xaxis': 'x9',
              'y': [encephalopathy  , mentions adams  , phillip adams  , suicide
                    case  , adams killed  ],
              'yaxis': 'y9'},
             {'marker': {'col

In [123]:
model.merge_topics(comments_all, [-1, 18])

In [125]:
model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,1153,-1_suicide article_attempted suicide_suicides_...,"[suicide article, attempted suicide, suicides,...",[The article mentions that Harry Reid died aft...
1,0,613,0_suicide article_suicide_committed suicide_ps...,"[suicide article, suicide, committed suicide, ...",[The article mentions that the person who comm...
2,1,549,1_suicide article_suicide attempt_suicide rela...,"[suicide article, suicide attempt, suicide rel...","[The article mentions that Adam Lanza, the per..."
3,2,223,2_suicide prevention_discusses suicide_suicide...,"[suicide prevention, discusses suicide, suicid...",[The article discusses the impact of the COVID...
4,3,192,3_veteran suicide_suicides among_suicide among...,"[veteran suicide, suicides among, suicide amon...",[The article mentions that high suicide rates ...
5,4,139,4_covid 19_coronavirus pandemic_suicides_19 pa...,"[covid 19, coronavirus pandemic, suicides, 19 ...","[The article mentions Heidi Ferrer, a woman wh..."
6,5,102,5_drug overdose_overdoses_cause suicide_opioid...,"[drug overdose, overdoses, cause suicide, opio...",[The article mentions a woman named Stephanie ...
7,6,97,6_transgender youth_transgender children_trans...,"[transgender youth, transgender children, tran...",[The article mentions that Haley Gabriella Fel...
8,7,92,7_assisted suicide_assisted death_euthanasia_e...,"[assisted suicide, assisted death, euthanasia,...",[The article discusses the issue of terminally...
9,8,85,8_adams killed_suicide case_phillip adams_ment...,"[adams killed, suicide case, phillip adams, me...",[The article mentions that the former NFL play...


In [137]:
topics = model.topics_
probs = model.probabilities_

# from existing predictions
left = data['Left'].to_list()
indsleft = []
for i in range(len(comments_all)):
    if left[i] == 1:
        indsleft.append(i)

alltopicsleft = [topics[i] for i in indsleft if i < len(topics)]
allprobsleft = [probs[i] for i in indsleft if i < len(probs)]
allcommentsleft = [comments_all[i] for i in indsleft if i < len(comments_all)]
onetextleft = " ".join(allcommentsleft)

right = data['Right'].to_list()
indsright = []
for i in range(len(comments_all)):
    if right[i] == 1:
        indsright.append(i)

alltopicsright = [topics[i] for i in indsright if i < len(topics)]
allprobsright = [probs[i] for i in indsright if i < len(probs)]
allcommentsright = [comments_all[i] for i in indsright if i < len(comments_all)]
onetextright = " ".join(allcommentsright)

regional = data['Regional'].to_list()
indsreg = []
for i in range(len(comments_all)):
    if regional[i] == 1:
        indsreg.append(i)

alltopicsreg = [topics[i] for i in indsreg if i < len(topics)]
allprobsreg = [probs[i] for i in indsreg if i < len(probs)]
allcommentsreg = [comments_all[i] for i in indsreg if i < len(comments_all)]
onetextreg = " ".join(allcommentsreg)

national = data['National'].to_list()
indsnat = []
for i in range(len(comments_all)):
    if national[i] == 1:
        indsnat.append(i)

alltopicsnat = [topics[i] for i in indsnat if i < len(topics)]
allprobsnat = [probs[i] for i in indsnat if i < len(probs)]
allcommentsnat = [comments_all[i] for i in indsnat if i < len(comments_all)]
onetextnat = " ".join(allcommentsnat)

embeddingsleft = sentence_model.encode(onetextleft, show_progress_bar=True)
embeddingsright = sentence_model.encode(onetextright, show_progress_bar=True)
embeddingsnat = sentence_model.encode(onetextnat, show_progress_bar=True)
embeddingsreg = sentence_model.encode(onetextreg, show_progress_bar=True)


onetopicleft, oneprobleft = model.transform(onetextleft)
onetopicright, oneprobright = model.transform(onetextright)
onetopicreg, oneprobreg = model.transform(onetextreg)
onetopicnat, oneprobnat = model.transform(onetextnat)

topicwiseprobleft = np.array(allprobsleft).mean(axis = 0)
topicwiseprobright = np.array(allprobsright).mean(axis = 0)
topicwiseprobnat = np.array(allprobsnat).mean(axis = 0)
topicwiseprobreg = np.array(allprobsreg).mean(axis = 0)

np.set_printoptions(suppress=True, formatter={'float': '{:0.5f}'.format})
print("Individual averaged: ", np.round(topicwiseprobleft - topicwiseprobright, 5))
print("Combined 1 text: ", np.round(oneprobleft - oneprobright, 5))

print("Individual averaged: ", np.round(topicwiseprobnat - topicwiseprobreg, 5))
print("Combined 1 text: ", np.round(oneprobnat - oneprobreg, 5))

Batches: 100%|██████████| 1/1 [00:00<00:00,  1.71it/s]


Individual averaged:  [0.02164 -0.02102 0.01087 -0.01749 0.00105 -0.00728 -0.00348 -0.01065
 0.00386 0.00267 0.00580 -0.00103 -0.01060 0.00312 -0.00912 0.00286
 0.00082 -0.00008 0.00034 0.00216 0.00629 0.00180 0.00362 0.00519 0.00000
 0.00364 0.00065 -0.00095 -0.00455 0.00323 0.00233 0.00049 0.00006 0.00361]
Combined 1 text:  [[-0.30400 -0.07416 -0.05127 -0.02187 -0.01245 -0.00875 -0.00103 -0.00933
  0.30695 -0.01345 -0.00640 -0.00149 -0.00507 -0.00290 -0.00641 -0.00458
  -0.00176 -0.00136 -0.00223 -0.00087 -0.00152 -0.00142 -0.00148 -0.00133
  -0.00072 -0.00137 -0.00212 -0.00153 -0.00123 -0.00186 -0.00227 -0.00215
  -0.00265 -0.00270]]
Individual averaged:  [-0.02387 0.01268 -0.02341 -0.00857 0.01268 -0.00710 0.00846 -0.00067
 0.00479 0.00883 0.00181 -0.00162 -0.00144 0.00213 -0.00259 -0.00368
 0.00087 -0.00697 0.00028 0.00322 0.00531 0.00342 0.00109 0.00437 -0.00639
 0.00098 -0.00438 -0.00329 -0.00205 0.00208 0.00007 0.00250 -0.00472
 0.00176]
Combined 1 text:  [[-0.04179 -0.03728 -0

In [138]:
df = model.get_topic_info()

topicwiseprobl = np.insert(topicwiseprobleft, 0,-1)
topicwiseprobr = np.insert(topicwiseprobright, 0,-1)
topicwiseprobnat = np.insert(topicwiseprobnat, 0,-1)
topicwiseprobreg = np.insert(topicwiseprobreg, 0,-1)
df['Left - Individual Averaged'] = topicwiseprobl
df['Right - Individual Averaged'] = topicwiseprobr
df['National - Individual Averaged'] = topicwiseprobnat
df['Regional - Individual Averaged'] = topicwiseprobreg

oneprobl = np.insert(oneprobleft, 0,-1)
oneprobr = np.insert(oneprobright, 0,-1)
oneprobn = np.insert(oneprobnat, 0,-1)
oneprobre = np.insert(oneprobreg, 0,-1)
df['Left - One combined text'] = oneprobl
df['Right - One combined text'] = oneprobr
df['National - One combined text'] = oneprobn
df['Regional - One combined text'] = oneprobre

df['Left v/s Right - Individual Averaged'] = topicwiseprobl - topicwiseprobr
df['National v/s Regional - Individual Averaged'] = topicwiseprobnat - topicwiseprobreg
df['Left v/s Right - Combined Text'] = oneprobl - oneprobr
df['National v/s Regional - Combined Text'] = oneprobn - oneprobre

df.to_csv('health betopic results.csv')

In [133]:
ben = data.copy()
probabilities = model.probabilities_
print(probabilities.shape)
for i in range(probabilities.shape[1]):
    ben['Topic' + str(i)] = probabilities[:, i]
ben.columns


(3756, 34)


Index(['Unnamed: 0', 'id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem', 'None',
       'Reasoning', 'Only Reason', 'National', 'Regional', 'Left', 'Right',
       'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23',
       'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29',
       'Topic30', 'Topic31', 'Topic32', 'Topic33'],
      dtype='object')

In [134]:
ben = ben.drop(columns=['Unnamed: 0', 'Reasoning'])

In [135]:
ben.columns

Index(['id', 'title', 'content', 'Publisher', 'Date Published',
       'Financial/Job Problem', 'School or Academic Related Problem',
       'Lack of Access to Health/Mental Health Care', 'Legal Problem', 'None',
       'Only Reason', 'National', 'Regional', 'Left', 'Right',
       'Fringe/Questionable', 'Topic0', 'Topic1', 'Topic2', 'Topic3', 'Topic4',
       'Topic5', 'Topic6', 'Topic7', 'Topic8', 'Topic9', 'Topic10', 'Topic11',
       'Topic12', 'Topic13', 'Topic14', 'Topic15', 'Topic16', 'Topic17',
       'Topic18', 'Topic19', 'Topic20', 'Topic21', 'Topic22', 'Topic23',
       'Topic24', 'Topic25', 'Topic26', 'Topic27', 'Topic28', 'Topic29',
       'Topic30', 'Topic31', 'Topic32', 'Topic33'],
      dtype='object')

In [136]:
ben.to_csv('Health problem articles topic model results.csv')

In [150]:
# leftarticles = data[data['Left'] == 1]
# rightarticles = data[data['Right'] == 1]

# topicsleft, probsleft = model.transform(leftarticles['Only Reason'].to_list())
# topicsright, probsright = model.transform(rightarticles['Only Reason'].to_list())

# print(np.round(np.array(probsleft).mean(axis = 0) - np.array(probsright).mean(axis = 0), 5))

[0.00978 -0.01605 0.00355 -0.00749 -0.00661 -0.00545 -0.00365 -0.01803
 -0.00079 0.00228 -0.00010 0.00225 -0.00085 0.01250 -0.00011 0.00285
 -0.00572 -0.00225 0.00001 0.00000 0.00081 0.00073 0.00129 0.00361 0.00531
 0.00323 -0.00150 0.00192 0.00444 0.00143 0.00484 0.00384 0.00001 0.00344
 0.00602 0.00002 0.00313]
